In [1]:
import numpy
import math
import pandas
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from sklearn.utils import check_arrays

from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout

Using TensorFlow backend.


In [2]:
numpy.random.seed(7)

dataframe = pandas.read_csv('Paper66_10min.csv', header=None)
dataframe = dataframe.iloc[:, [1,2,4,5,6,7,9,11]]
dataset = dataframe.values
dataset = dataset.astype('float32')

#summary of data
Data_Summary = numpy.zeros((8,3))

for i in range (len(dataset[0, :])):
    Data_Summary[i,0] = numpy.max(dataset[:, i])
    Data_Summary[i,1] = numpy.mean(dataset[:, i])
    Data_Summary[i,2] = numpy.std(dataset[:, i])

print (Data_Summary)

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

[[ 1.77149999  0.19888213  0.10472103]
 [ 1.13699996  0.12350018  0.13087003]
 [ 1.14999998  0.23069003  0.27148944]
 [ 0.74599999  0.06392039  0.06839383]
 [ 1.14300001  0.55704874  0.37750435]
 [ 3.29500008  1.46453643  1.03900933]
 [ 0.96950001  0.17980404  0.17077641]
 [ 1.6595      0.27750552  0.35760996]]


In [3]:
#train, test split
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [4]:
def create_dataset(dataset, look_back, step):
    dataX, dataY = [], []
    
    for i in range(len(dataset)-look_back-step): #i是矩阵的列数
        a = numpy.reshape(dataset[i:(i+look_back), :], -1)
        dataX.append(a)
        b = numpy.reshape(dataset[i+look_back+step-1, :], -1)
        dataY.append(b)
    
    return numpy.array(dataX), numpy.array(dataY)

#Nash-Sutcliffe
def NSE(observed, simulated):
    return 1 - numpy.sum((simulated-observed)**2)/numpy.sum((observed-numpy.mean(observed))**2)

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
look_back = 48
step = 8

TrainX, TrainY = create_dataset(train, look_back, step)
TestX, TestY = create_dataset(test, look_back, step)

In [ ]:
TrainX = numpy.reshape(TrainX, (TrainX.shape[0], 1, TrainX.shape[1]))
TestX = numpy.reshape(TestX, (TestX.shape[0], 1, TestX.shape[1]))

LSTMNN = Sequential()
LSTMNN.add(LSTM(512, return_sequences=True, input_dim=8*look_back))
LSTMNN.add(Dropout(0.2))
LSTMNN.add(LSTM(512))
LSTMNN.add(Dense(8))
LSTMNN.compile(loss='mean_squared_error', optimizer='adam')
LSTMNN.fit(TrainX, TrainY, nb_epoch=60, batch_size=1024, verbose=2)

LSTMpredict = LSTMNN.predict(TestX)

LSTMpredict = scaler.inverse_transform(LSTMpredict)
TestY = scaler.inverse_transform(TestY)

Epoch 1/60
46s - loss: 0.0225
Epoch 2/60


In [ ]:
plt.plot(LSTMpredict[:,0],'b--')
plt.plot(TestY[:,0],'r:',markersize=3)

plt.show()

In [ ]:
plt.plot(LSTMpredict[:,3],'b--')
plt.plot(TestY[:,3],'r:',markersize=3)

plt.show()

In [ ]:
plt.plot(LSTMpredict[:,7],'b--')
plt.plot(TestY[:,7],'r:',markersize=3)

plt.show()

In [ ]:
Model_Result = numpy.zeros((8,3))

for i in range (len(dataset[0, :])):
    Model_Result[i,0] = numpy.corrcoef(TestY[:,i], LSTMpredict[:,i])[0, 1]
    Model_Result[i,1] = numpy.sqrt(mean_squared_error(TestY[:,i], LSTMpredict[:,i]))
    Model_Result[i,2] = NSE(TestY[:,i], LSTMpredict[:,i])
    #Model_Result[i,2] = mean_absolute_percentage_error(TestY[:,i], LSTMpredict[:,i])

print (Model_Result)